In [607]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

df = pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv')
print(df.shape)
df.head(5)

(1112, 13)


,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr
0,1,50002,1,50002,PITT,no_filename,1,10.201539,1.194664,16.223458,3.878000,0.152711,12.072452
1,2,50003,2,50003,PITT,Pitt_0050003,1,7.165701,1.126752,10.460008,4.282238,0.161716,9.241155
2,3,50004,3,50004,PITT,Pitt_0050004,1,7.698144,1.226218,9.725750,3.881684,0.174186,9.323463
3,4,50005,4,50005,PITT,Pitt_0050005,1,9.071807,1.256278,11.198226,3.628667,0.119269,10.814200
4,5,50006,5,50006,PITT,Pitt_0050006,1,8.026798,1.407166,6.282055,3.674539,0.130647,10.123574


In [608]:
X = np.load("features.npz")['a']
y = np.load("labels.npz")['a']
y = np.select([y == 1, y == 2], [0, 1], y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

print("\nTraining data samples:")
print(X_train.shape)


Training data samples:
(583, 2016)


In [609]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#y_train =  torch.from_numpy(y_train.ravel()).float()
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

print(y_train)
print(X_test)

tensor([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
        0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
        0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 

In [610]:
#Function to train data
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Calling dataset class to train on X and y tensors
train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
#Function to test data
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
#Calling dataset class to test on X tensor
test_data = TestData(torch.FloatTensor(X_test))

In [611]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [612]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(2016, 256) 
        self.layer_2 = nn.Linear(256, 50)
        self.layer_out = nn.Linear(50, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(256)
        self.batchnorm2 = nn.BatchNorm1d(50)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [613]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


cpu
BinaryClassification(
  (layer_1): Linear(in_features=2016, out_features=256, bias=True)
  (layer_2): Linear(in_features=256, out_features=50, bias=True)
  (layer_out): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [614]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [615]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Train Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Train Loss: 0.72079 | Acc: 55.300
Epoch 002: | Train Loss: 0.57350 | Acc: 68.100
Epoch 003: | Train Loss: 0.50200 | Acc: 78.000
Epoch 004: | Train Loss: 0.38745 | Acc: 86.500
Epoch 005: | Train Loss: 0.37216 | Acc: 86.700
Epoch 006: | Train Loss: 0.29210 | Acc: 90.800
Epoch 007: | Train Loss: 0.28814 | Acc: 91.100
Epoch 008: | Train Loss: 0.21501 | Acc: 94.700
Epoch 009: | Train Loss: 0.18707 | Acc: 94.800
Epoch 010: | Train Loss: 0.12617 | Acc: 98.100
Epoch 011: | Train Loss: 0.11774 | Acc: 97.700
Epoch 012: | Train Loss: 0.11431 | Acc: 97.500
Epoch 013: | Train Loss: 0.13507 | Acc: 96.400
Epoch 014: | Train Loss: 0.15640 | Acc: 96.800
Epoch 015: | Train Loss: 0.16032 | Acc: 96.200
Epoch 016: | Train Loss: 0.18632 | Acc: 92.500
Epoch 017: | Train Loss: 0.16223 | Acc: 95.100
Epoch 018: | Train Loss: 0.11925 | Acc: 96.500
Epoch 019: | Train Loss: 0.08198 | Acc: 98.000
Epoch 020: | Train Loss: 0.05249 | Acc: 100.000
Epoch 021: | Train Loss: 0.07302 | Acc: 97.900
Epoch 022: |

In [616]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        
    y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
#print(y_pred_list)    

In [617]:
model.eval()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Valid Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Valid Loss: 0.01988 | Acc: 99.400
Epoch 002: | Valid Loss: 0.01758 | Acc: 99.800
Epoch 003: | Valid Loss: 0.01384 | Acc: 100.000
Epoch 004: | Valid Loss: 0.00484 | Acc: 100.000
Epoch 005: | Valid Loss: 0.00280 | Acc: 100.000
Epoch 006: | Valid Loss: 0.00196 | Acc: 100.000
Epoch 007: | Valid Loss: 0.00178 | Acc: 100.000
Epoch 008: | Valid Loss: 0.00162 | Acc: 100.000
Epoch 009: | Valid Loss: 0.00139 | Acc: 100.000
Epoch 010: | Valid Loss: 0.00132 | Acc: 100.000
Epoch 011: | Valid Loss: 0.00126 | Acc: 100.000
Epoch 012: | Valid Loss: 0.00113 | Acc: 100.000
Epoch 013: | Valid Loss: 0.00109 | Acc: 100.000
Epoch 014: | Valid Loss: 0.00104 | Acc: 100.000
Epoch 015: | Valid Loss: 0.00100 | Acc: 100.000
Epoch 016: | Valid Loss: 0.00095 | Acc: 100.000
Epoch 017: | Valid Loss: 0.00086 | Acc: 100.000
Epoch 018: | Valid Loss: 0.00083 | Acc: 100.000
Epoch 019: | Valid Loss: 0.00079 | Acc: 100.000
Epoch 020: | Valid Loss: 0.00084 | Acc: 100.000
Epoch 021: | Valid Loss: 0.00073 | Acc: 10

In [618]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.64      0.70      0.67       132
         1.0       0.73      0.67      0.70       156

    accuracy                           0.68       288
   macro avg       0.68      0.69      0.68       288
weighted avg       0.69      0.68      0.68       288

